This script pulls in MLS player salary info for the 2018-2019 season via https://mlsplayers.org/resources/salary-guide

In [3]:
from bs4 import BeautifulSoup
from decimal import Decimal
import pandas as pd

In [4]:
with open("MLS_SALARIES.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [5]:
def scrape():
    teams = {}
    players = soup.findAll("tr")
    table = pd.DataFrame(columns={'first_name', 'last_name', 'team', 'position', 'base_pay', 'total_pay'})
    player_list = []
    for player in players[1:]:
        attributes = player.findAll("td")
        player_list.append({
            'first_name': str(attributes[0])[4:-5],
            'last_name': str(attributes[1])[4:-5],
            'team': str(attributes[2])[4:-5],
            'position': str(attributes[3])[4:-5],
            'base_pay': Decimal(str(attributes[4])[5:-5].replace(',', '')),
            'total_pay': Decimal(str(attributes[5])[5:-5].replace(',', ''))
        })
    return pd.DataFrame(player_list)

In [6]:
full_players = scrape()

In [60]:
temp_teams = {}
for index, player in full_players.iterrows():
    if player['team'] in temp_teams:
        temp_teams[player['team']]['total_base'] += player['base_pay']
        temp_teams[player['team']]['total_comp'] += player['total_pay']
        temp_teams[player['team']]['cost_per_player'] = round(temp_teams[player['team']]['total_comp'] / temp_teams[player['team']]['player_count'], 3)
        temp_teams[player['team']]['player_count'] += 1
    else:
        temp_teams[player['team']] = {
            'team': player['team'],
            'total_base': player['base_pay'],
            'total_comp': player['total_pay'],
            'player_count': 1
        }

In [75]:
teams = pd.DataFrame(temp_teams).transpose()
teams = teams[teams['player_count'] > 11]

KeyError: 'player_count'

In [62]:
teams = teams.reset_index(drop=True)

In [63]:
teams.iloc[0]

cost_per_player              412651.322
player_count                         29
team               Sporting Kansas City
total_base                  11132768.52
total_comp                  11554237.01
Name: 0, dtype: object

In [64]:
teams.sort_values('total_base', ascending=False)

,cost_per_player,player_count,team,total_base,total_comp
16,948543.095,29,Toronto FC,23811805.21,26559206.65
8,673346.473,27,LA Galaxy,14799179.79,17507008.30
20,575241.580,28,Chicago Fire,14701164.34,15531522.67
19,529442.528,29,New York City FC,13874057.71,14824390.78
2,504790.571,29,LAFC,11824368.73,14134135.99
0,412651.322,29,Sporting Kansas City,11132768.52,11554237.01
11,452986.473,27,Seattle Sounders FC,10677250.28,11777648.29
3,374397.758,32,Atlanta United,10669120.49,11606330.49
9,366493.099,31,Portland Timbers,10543917.96,10994792.96
12,358871.772,28,DC United,9192531.82,9689537.84
